In [8]:
import os
import string

if os.getcwd().endswith('notebooks'):
    os.chdir('..')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [10]:
import pandas as pd

In [1]:
import sys, os
os.path.dirname(sys.executable)

'/home/robert/tools/anaconda3/envs/thesis/bin'

In [70]:
train_set = pd.read_csv('data/interim/amazon_google_75/train.csv')
train_set.head()

,left_id,left_title,left_manufacturer,right_id,right_title,right_manufacturer,label
0,571,microsoft visio standard 2007 version upgrade,microsoft,946,adobe cs3 design standard upgrade,NaN,0
1,1174,adobe indesign cs3 upgrade from pagemaker [ mac ],adobe,946,adobe cs3 design standard upgrade,NaN,0
2,588,adobe creative suite cs3 design standard [ mac ],adobe,946,adobe cs3 design standard upgrade,NaN,0
3,518,microsoft office 2004 standard upgrade ( mac ),microsoft,946,adobe cs3 design standard upgrade,NaN,0
4,572,adobe dreamweaver cs3 upgrade,adobe,946,adobe cs3 design standard upgrade,NaN,0


In [38]:
from typing import Tuple, Set
import string


def differentiate(left_title: str, right_title: str) -> Tuple[Set[str], Set[str]]:
    left_token_set: Set[str] = set([word for word in left_title.split(' ') if word not in string.punctuation])
    right_token_set: Set[str] = set([word for word in right_title.split(' ') if word not in string.punctuation])

    common_tokens = left_token_set.intersection(right_token_set)
    left_only_tokens = left_token_set - right_token_set
    right_only_tokens = right_token_set - left_token_set

    return common_tokens, left_only_tokens.union(right_only_tokens)

In [121]:
train_set[train_set['label'] == 1]

,left_id,left_title,left_manufacturer,right_id,right_title,right_manufacturer,label,diff
21,571,microsoft visio standard 2007 version upgrade,microsoft,3027,microsoft office visio standard 2007 upgrade (...,NaN,1,"({microsoft, 2007, standard, visio, upgrade}, ..."
195,858,microsoft onenote 2007 upgrade,microsoft,2848,microsoft onenote 2007 upgrade ( pc ),NaN,1,"({microsoft, 2007, onenote, upgrade}, {pc})"
229,519,microsoft student with encarta premium 2008 dvd,microsoft,2423,microsoft student with encarta premium 2008 co...,NaN,1,"({microsoft, student, with, premium, 2008, enc..."
231,574,microsoft mappoint 2006 with gps,microsoft,3029,microsoft mappoint 2006 with gps locator ( pc ),NaN,1,"({microsoft, with, gps, 2006, mappoint}, {loca..."
251,225,microsoft powerpoint 2004 ( mac ),microsoft,2316,microsoft powerpoint 2004 software mac apple &...,NaN,1,"({microsoft, mac, powerpoint, 2004}, {apple, s..."
...,...,...,...,...,...,...,...,...
3305,612,corel ventura 10 dvd,corel,2299,ventura 10 dvd,NaN,1,"({10, ventura, dvd}, {corel})"
3306,491,hollywood volume 1,yamaha,2427,phillip keveren 501056 hollywood volume 1,NaN,1,"({volume, 1, hollywood}, {501056, phillip, kev..."
3307,1194,roxio toast 8 titanium,roxio,573,roxio toast 8 titanium cdr software,NaN,1,"({8, roxio, titanium, toast}, {cdr, software})"
3308,1291,big tiny little a tiny little christmas,yamaha,701,big tiny little big tiny little a tiny little ...,NaN,1,"({a, tiny, big, christmas, little}, {})"


In [157]:
import numpy as np


def extract_interesting_tokens(df: pd.DataFrame, label: int) -> pd.DataFrame:
    v = np.vectorize(differentiate)
    df = df[df['label'] == label].copy()

    result = v(df['left_title'], df['right_title'])
    result = [(result[0][i], result[1][i]) for i in range(len(result[0]))]
    df['diff'] = result

    interesting_set = pd.DataFrame()
    # (train_set['diff'].map(lambda x: len(x[0]) > len(x[1]))) &
    interesting_set['diff'] = df[df['diff'].map(lambda x: len(x[0]) > len([x[1]]))]['diff']
    interesting_set['key_tokens'] = interesting_set['diff'].apply(lambda x: x[1])
    interesting_set = interesting_set.explode('key_tokens')

    interesting_set = interesting_set.groupby('key_tokens').key_tokens.count().rename('count').reset_index()

    interesting_set['count_relative'] = interesting_set['count'] / len(df)
    interesting_set = interesting_set.sort_values('count_relative', ascending=False)

    return interesting_set

In [158]:
negative_tokens = extract_interesting_tokens(train_set, 0)
negative_tokens.head(20)

,key_tokens,count,count_relative
1286,software,527,0.184266
908,mac,478,0.167133
701,for,394,0.137762
1439,upgrade,334,0.116783
1340,suite,307,0.107343
555,design,247,0.086364
1117,premium,233,0.081469
1315,standard,229,0.080070
1519,windows,214,0.074825
1132,pro,207,0.072378


In [159]:
neutral_tokens = extract_interesting_tokens(train_set, 1)
neutral_tokens.head(20)

,key_tokens,count,count_relative
595,software,135,0.298013
698,win,49,0.108168
423,mac,48,0.105960
311,encore,47,0.103753
338,for,42,0.092715
500,pc,40,0.088300
81,2000,40,0.088300
707,xp,37,0.081678
373,inc,33,0.072848
142,98,29,0.064018
